# Building MIMO systems

This notebook describes how to define MIMO systems in the `python-control` library. We'll use the Wood-Berry (1973) distillation model as an example for a MIMO system.

*Note: You'll need `slycot` installed to work with MIMO systems in `python-control`.*

Details of the model is available in **Loquasto and Seborg (2013) [1]**:

> The Wood-Berry model is a well-known 2 × 2 transfer function model of a pilot-plant distillation column for a methanol-water mixture. The output variables are the distillate and bottoms compositions, $x_D$ and $x_B$ [weight % methanol]. They are controlled by manipulating the reflux and steam flow rates, $R$ and $S$ [lb/min]. The feed flow rate, $F$, is an unmeasured disturbance variable.


References:

1. See Case Study 5: [Loquasto, Fred, and Dale E. Seborg. "Monitoring model predictive control systems using pattern classification and neural networks." *Industrial & engineering chemistry research* 42.20 (2003): 4689-4701.](http://www.chemengr.ucsb.edu/~ceweb/faculty/seborg/pdfs/Loquasto_IEC.pdf)

As you can see in the paper, the Wood-Berry MIMO model is given in the form of a transfer function matrix:

$
\begin{bmatrix}
x_D(s) \\
x_B(s) 
\end{bmatrix}= 
\begin{bmatrix}
G_{11} & G_{12} \\
G_{21} & G_{22} 
\end{bmatrix}
\begin{bmatrix}
R(s) \\
S(s) 
\end{bmatrix}+
\begin{bmatrix}
G_{1F} \\
G_{2F} 
\end{bmatrix}
F(s)
$

Here's how we define a MIMO system in the `control` library:

In [13]:
import control

### 1. Define the transfer function using its coefficients
Let's start with $G_{11}$. Here's the transfer function without a time delay using `control.tf`. Easy:

In [14]:
g11_tf = control.tf([12.8], [16.7,1])
g11_tf


   12.8
----------
16.7 s + 1

### 2. Add the time delay
We'll use a first-order Pade approximation using `control.pade`.

`control.pade` returns 2 outputs for the approximation, the coefficients of the (1) numerator and the (2) denominator. We'll use that to generate a transfer function with `control.tf`:

In [15]:
delay = 1
num, deg = control.pade(delay)
control.tf(num, deg)


-s + 2
------
s + 2

### 3. Combine the time delay with the transfer function
We can now multiply `g11_tf` with the delay approximation to get the final transfer function for $G_{11}$:

In [16]:
g11 = g11_tf*control.tf(num, deg)
g11


    -12.8 s + 25.6
---------------------
16.7 s^2 + 34.4 s + 2

### 4. Now repeat for all the other transfer functions $G_{12}$, $G_{21}$, $G_{22}$ etc.:

In [17]:
g_tf = control.tf([12.8], [16.7,1])
delay = 1
num, deg = control.pade(delay)
g11 = g_tf*control.tf(num, deg)

g_tf = control.tf([-18.9], [21.0,1])
delay = 3
num, deg = control.pade(delay)
g12 = g_tf*control.tf(num, deg)

g_tf = control.tf([6.6], [10.9,1])
delay = 7
num, deg = control.pade(delay)
g21 = g_tf*control.tf(num, deg)

g_tf = control.tf([-19.4], [14.4,1])
delay = 3
num, deg = control.pade(delay)
g22 = g_tf*control.tf(num, deg)

g_tf = control.tf([3.8], [14.9,1])
delay = 8
num, deg = control.pade(delay)
g1f = g_tf*control.tf(num, deg)

g_tf = control.tf([4.9], [13.2,1])
delay = 3
num, deg = control.pade(delay)
g2f = g_tf*control.tf(num, deg)

### 5. Bring it all together into one MIMO system

Now that we have all the transfer functions, we need to piece it together into a transfer function matrix this form using `control.tf`:

\begin{bmatrix}
G_{11} & G_{12} \\
G_{21} & G_{22} 
\end{bmatrix}

According to the documentation:

> "To create a MIMO system, num and den need to be 2D nested lists of array_like objects"

This means that we need to split the transfer functions into the coefficients of their numerators and denominators and created a nested array of `[numerators, denominators]`. 

- Each element in `numerators` contains an array of coefficients for a particular row in the TF matrix, i.e. `numerators` = [num_row1, num_row2].
- Whereas each element in the `num_row` arrays contains the coefficients of a particular TF in that row, i.e. `num_row1` = coefficients of g11.
- Same goes for `denominators`.

In other words, our MIMO system needs to be defined as follows:

```python
control.tf([[
             [G11_num, G12_num],
             [G21_num, G22_num],
            ],
            [
             [G11_den, G12_den],
             [G21_den, G22_den],            
            ]])
```

Let's start with an example for $G_{11}$:

In [18]:
g11.num

[[array([-12.8,  25.6])]]

Notice that the `control` library returns doubly nested array of coefficients. We'll need to dive 2 layers into the array to grab the coefficients of one coefficient in one array:

In [19]:
g11.num[0][0]

array([-12.8,  25.6])

Now let's do the same thing for the other transfer functions. We'll use list comprehension to group the coefficients of one row together. 

To obtain `[G11_num, G12_num]`, we would do:

In [20]:
[x[0][0] for x in (g11.num, g12.num)]

[array([-12.8,  25.6]), array([ 18.9, -12.6])]

In [21]:
row_1_num = [x[0][0] for x in (g11.num, g12.num)]
row_2_num = [x[0][0] for x in (g21.num, g22.num)]
row_1_den = [x[0][0] for x in (g11.den, g12.den)]
row_2_den = [x[0][0] for x in (g21.den, g22.den)]

Now it's easy to combine them together to get our MIMO system:

In [22]:
sys = control.tf(
    [row_1_num,row_2_num],
    [row_1_den,row_2_den]
)
sys


Input 1 to output 1:
    -12.8 s + 25.6
---------------------
16.7 s^2 + 34.4 s + 2

Input 1 to output 2:
      -6.6 s + 1.886
---------------------------
10.9 s^2 + 4.114 s + 0.2857

Input 2 to output 1:
    18.9 s - 12.6
----------------------
21 s^2 + 15 s + 0.6667

Input 2 to output 2:
      19.4 s - 12.93
--------------------------
14.4 s^2 + 10.6 s + 0.6667

#### Section reserved (TODO)

I need to figure out how to make linear combinations of TF matrices in the `control` library.

---


### 5. Alternative method by rearranging the equation

What about the second term with the Feed? With some simple algebraic manipulation, we can see that our original equation is equivalent to a $3\times2$ MIMO system:

$
\begin{bmatrix}
x_D(s) \\
x_B(s) 
\end{bmatrix}= 
\begin{bmatrix}
G_{11} & G_{12} & G_{1F}\\
G_{21} & G_{22} & G_{2F}
\end{bmatrix}
\begin{bmatrix}
R(s) \\
S(s) \\
F(s)
\end{bmatrix}
$

Knowing that, we can define our final MIMO system as follows:

In [23]:
g_tf = control.tf([12.8], [16.7,1])
td = 1
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g11 = g_tf*g_delay

g_tf = control.tf([-18.9], [21.0,1])
td = 3
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g12 = g_tf*g_delay

g_tf = control.tf([6.6], [10.9,1])
td = 7
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g21 = g_tf*g_delay

g_tf = control.tf([-19.4], [14.4,1])
td = 3
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g22 = g_tf*g_delay

g_tf = control.tf([3.8], [14.9,1])
td = 8
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g1f = g_tf*g_delay

g_tf = control.tf([4.9], [13.2,1])
td = 3
g_delay = control.tf(control.pade(td,1)[0],control.pade(td,1)[1])
g2f = g_tf*g_delay

row_1_num = [x[0][0] for x in (g11.num, g12.num, g1f.num)]
row_2_num = [x[0][0] for x in (g21.num, g22.num, g2f.num)]

row_1_den = [x[0][0] for x in (g11.den, g12.den, g1f.den)]
row_2_den = [x[0][0] for x in (g21.den, g22.den, g2f.den)]

sys = control.tf(
    [row_1_num,row_2_num],
    [row_1_den,row_2_den]
)

sys


Input 1 to output 1:
    -12.8 s + 25.6
---------------------
16.7 s^2 + 34.4 s + 2

Input 1 to output 2:
      -6.6 s + 1.886
---------------------------
10.9 s^2 + 4.114 s + 0.2857

Input 2 to output 1:
    18.9 s - 12.6
----------------------
21 s^2 + 15 s + 0.6667

Input 2 to output 2:
      19.4 s - 12.93
--------------------------
14.4 s^2 + 10.6 s + 0.6667

Input 3 to output 1:
      -3.8 s + 0.95
-------------------------
14.9 s^2 + 4.725 s + 0.25

Input 3 to output 2:
      -4.9 s + 3.267
-------------------------
13.2 s^2 + 9.8 s + 0.6667

## Takeaways:

By the end of this notebook, you should be able to:

1. Use `control.pade` to create a rational transfer function approximation for a time delay
2. Build a MIMO system using `control.tf` by combining individual transfer functions to form a TF matrix